# Decorators written as classes

In [40]:
from unittest.mock import MagicMock

In [15]:
import random

class trace_it:
    def __init__(self, func):
        self.func = func
    def __call__(self, *args, **kwargs):
        print(args, kwargs)
        return self.func(*args, **kwargs)
        
@trace_it
def rand(min_val, max_val):
    return random.randint(min_val, max_val)
rand(10,30)

(10, 30) {}


15

# Categories
Category terminology propsed by Andy Fundinger from Bloomberg.

## A-Decorators (Argument Changing)
* add or remove an argument when the function is called
* change the value or type of an argument at call time
* similarly alter the return value
**Problems**
* calling the apparent signature does not actually work
* calling a function for a test requires injecting data to drive the decorator properly

### Example: pytest.mark.parametrize()

In [19]:
import pytest

@pytest.mark.parametrize("test_input,expected", [
    ("3+5", 8),
    ("2+4", 6),
    ("6*9", 42),
])

def test_eval(test_input, expected):
    assert eval(test_input) == expected

## B-Decorators (Binding Decorators)
* implement the Descriptor Protocol to change how functions behave
* the standard library includes @staticmethod, @classmethod, and @property

**Problems**
* creates an alternative to instance methods and attributes
* new language patterns arguably better fitting other languages
*time shifts otherwise normal exceptions or introduces new ones

### Example: SQLAlchemy Hybrid Properties

In [39]:
#from sqlalchemy import func
hybrid_property= MagicMock()

class Interval(object):
    @hybrid_property
    def radius(self):
        return abs(self.length) / 2

    @radius.expression
    def radius(cls):
        return func.abs(cls.length) / 2

### Example implementation -- instance method

In [27]:
class instance_method:
    def __init__(self, func):
        self.func = func
    def __get__(self, inst, cls):
        if inst is None:
            raise TypeError(f'{self.func.__name__} is only valid on instances.')
        return self.func.__get__(inst, cls)
    
class GoodClass:
    @instance_method
    def simple_method(self):
        print('simple_method')
    def normal_method(self):
        print('normal_method')

In [28]:
GoodClass().normal_method()
GoodClass().simple_method()

normal_method
simple_method


In [30]:
GoodClass.normal_method

<function __main__.GoodClass.normal_method(self)>

In [31]:
GoodClass.simple_method

TypeError: simple_method is only valid on instances.

## C-Decorators (Control Flow)
* change whether a function will be called and how many times

**Problems**
* a predictable control flow now has a hidden conditional
* a single invocation might now lead to 0, 1, or many executions of the function

### Example: Retry Decorator

In [34]:
#from retrying import retry
retry = MagicMock()

@retry
def do_something_unreliable():
    if random.randint(0, 10) > 1:
        raise IOError("Broken sauce, everything is hosed!!!111one")
    else:
        return "Awesome sauce!"

print(do_something_unreliable())

<MagicMock name='mock()()' id='140608917228344'>


### Example implementation -- infinite retry

In [37]:
def infinite_retry(func):
    def wrapper(*args, **kwargs):
        while True:
            try:
                return func(*args, **kwargs)
            except RuntimeError as e:
                print(e)
    return wrapper

@infinite_retry
def random_fail(max_value):
    ret = random.randint(-100, max_value)
    if ret<0:
        raise RuntimeError("Invalid negative number {ret}".format(ret=ret))
    return ret

random_fail(10)

Invalid negative number -65
Invalid negative number -69
Invalid negative number -24


0

In [38]:
#import pytest
pytest = MagicMock()

@pytest.mark.webtest
def test_send_http():
    pass # perform some webtest test for your app
def test_something_quick():
    pass
def test_another():
    pass
class TestClass(object):
    def test_method(self):
        pass

## D-Decorators (Descriptive)
* add the decorated object to some sort of collection
* this collection will serve some other purpose such as:
 * documentation
 * dispatching
 * plugins

**Problems**
* it's unclear how dispatching will be done as a result of registration
* similarly it's hard to see where the registration is maintained

### Example: flask.app.route

In [43]:
#app = Flask(__name__)
app = MagicMock()

@app.route('/')
def hello_world():
    return 'Hello, World!'

### Example implementation -- qa list

In [46]:
import warnings 

to_qa=[]
def qa(func):
    to_qa.append(func.__name__)
    return func

@qa
def new_code(): pass

@qa
def refactored_code(): pass

def well_trusted_code(): pass

In [47]:
to_qa

['new_code', 'refactored_code']

## E-Decorators (Execution)
* reads the method/class code
* may reinterpret the source code to basically not be python

**Problems**
* many
* and more

This truly means that the code you wrote is changed--by the decorator--to some other code that is then executed. It might:

* be analyzed for dependencies
* have objects in the ast swapped out, injected or removed
* be recompiled with different rules

### Example: cython

In [50]:
cython = MagicMock()

@cython.locals(a=cython.double, b=cython.double, n=cython.p_double)
def foo(a, b, x, y):
    n = a*b
    ...

### Example implementation -- code replacer

In [57]:
import inspect
def replacer(old, new):
    def deco(func):
        source = inspect.getsource(func.__code__)
        lines = source.split('\n')
        new_source = lines[1]+'\n'+('\n'.join(lines[2:]).replace(old, new))
        exec(new_source,globals())
        return globals()[func.__name__]
    return deco

In [55]:
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sample: 10


In [59]:
@replacer('b','b*3')
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sample: 26


**Careful...**

In [62]:
@replacer('a','a*3')
def sample(a, b):
    x = a + b
    y = x * 2
    print('Sample: ' + str(y))
sample(1,4)

Sa*3mple: 14


Other tools available for execution decorators
* bytecode manipulation
* ast manipulation

# Examples

* Argument changing
 * @click.option
 * @flask.templated
 * @django.views.decorators.gzip.gzip_page
* Binding
 * @variants.primary
 * @pyramid.decorator.reify
* Control flow
 * @functools.lru_cache
 * @django.views.decorators.http.require_http_methods
 * @twisted.internet.defer.inlineCallbacks
* Descriptive
 * @numpy.testing.decorators.setastest
* Execution
 * @numba.jit